# Klein-Gordon Equation

## Start with wave equation

Since Klein-Gordon equation has one more term than wave equation, and it is also a linear equation, so let us introduce wave equation first.

We have learned the linear one-dimensional diffusion equation:

\begin{equation}\frac{\partial u}{\partial t}= \nu \frac{\partial^2 u}{\partial x^2}\end{equation}

We now know how to deal with the second-order derivative in space. The finite-difference method of Central Difference Scheme was introduced to solve it.

Everyone should be familar with the wave equation:

\begin{equation}\frac{\partial^2 u}{\partial t^2}= c^2 \frac{\partial^2 u}{\partial x^2}\end{equation}

Beacuse we know that the time should be always forward, should we use the same method of Cetral Difference Scheme to derivative the second-order in time?

## CTCS

The answer of the question above is yes! Since we are not quite sure about the CTCS scheme now, let us try to get this in another way.

We are going to solve the wave equation by the solution of coupled first-order PDEs. Now, let us rewrite wave equation.

So, we set:

\begin{equation}\ r= c \frac{\partial u}{\partial x}\end{equation}

\begin{equation}\ s=  \frac{\partial u}{\partial t}\end{equation}

which means wave equation can be rewrited as:

\begin{equation}\frac{\partial r}{\partial t}= c \frac{\partial s}{\partial x}\end{equation}

\begin{equation}\frac{\partial s}{\partial t}= c \frac{\partial r}{\partial x}\end{equation}

\begin{equation}\frac{\partial u}{\partial t}= s\end{equation}

Let's set U and F as two vectors as what we have learned in moduel 1. 

\begin{align}
U & = \begin{pmatrix} r \\ s \end{pmatrix} & F(U) & = \begin{pmatrix} 0 & -c \\ -c & 0 \end{pmatrix} U \nonumber
\end{align}

And then we can get:

\begin{equation}\frac{\partial U}{\partial t}= - \frac{\partial F(U)}{\partial x}\end{equation}

The leapfrog scheme was introuduced in the class, since we get a first-order PDE now, we are going to try leapfrog scheme with it.

\begin{equation}r_{i+1/2}^{n+1} = r_{i+1/2}^{n} + \frac{\Delta t}{\Delta x}(s^{n+1/2}_{i+1}- s^{n+1/2}_{i}) + {\mathcal O}(\Delta x^2) \end{equation}

\begin{equation}s_{i}^{n+1/2} = s_{i}^{n-1/2} + \frac{\Delta t}{\Delta x}(r^{n}_{i+1/2}- r^{n}_{i-1/2})  + {\mathcal O}(\Delta x^2) \end{equation}

And let us put these two equation into above

\begin{equation}r_{i+1/2}^{n} = v(u^n_{i+1}- u^n_{i})/\Delta x + {\mathcal O}(\Delta x) \end{equation}

\begin{equation}s_{i}^{n+1/2} = (u^{n+1}_i- u^n_{i})/\Delta t  + {\mathcal O}(\Delta t) \end{equation}

Right now we can get the representation of wave equation with leapfrog scheme:

\begin{equation}\frac{u_{i}^{n+1}-2u_{i}^{n}+u_{i}^{n-1}}{\Delta t^2}=c^2\frac{u_{i+1}^{n}-2u_{i}^{n}+u_{i-1}^{n}}{\Delta x^2}\end{equation}

Good! This is the second-order differencing of wave equation, and this is also the CTCS.

Here is the stencil for CTCS.

